In [64]:
import os
import pandas as pd
import sql as gd

In [75]:
flights_faso = gd.get_data('SELECT * FROM flights_faso')
flights_faso.head()

/opt/homebrew/Caskroom/miniconda/base/envs/sql-practice/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,flight_date,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,airline,tail_number,flight_number,origin,dest,air_time,distance,cancelled,diverted
0,2012-10-01,933.0,900,33.0,1305.0,1205,60.0,AA,N320AA,1,JFK,LAX,317.0,2475.0,0,0
1,2012-10-02,906.0,900,6.0,1200.0,1205,-5.0,AA,N319AA,1,JFK,LAX,328.0,2475.0,0,0
2,2012-10-03,900.0,900,0.0,1225.0,1205,20.0,AA,N319AA,1,JFK,LAX,340.0,2475.0,0,0
3,2012-10-04,853.0,900,-7.0,1205.0,1205,0.0,AA,N325AA,1,JFK,LAX,336.0,2475.0,0,0
4,2012-10-05,853.0,900,-7.0,1236.0,1205,31.0,AA,N325AA,1,JFK,LAX,368.0,2475.0,0,0


In [76]:
# getting lat and lon of selected airports from airports table in database

points = gd.get_data("SELECT faa, lat,lon FROM airports WHERE faa in ('JFK', 'IAD', 'LGA', 'DCA', 'PHX', 'ORD', 'LAX')")
points
         


/opt/homebrew/Caskroom/miniconda/base/envs/sql-practice/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,faa,lat,lon
0,PHX,33.434299,-112.012001
1,LAX,33.942501,-118.407997
2,DCA,38.852100,-77.037697
3,LGA,40.777199,-73.872597
4,IAD,38.944500,-77.455803
5,JFK,40.639801,-73.778900
6,ORD,41.978600,-87.904800


In [46]:
import requests

url = "https://meteostat.p.rapidapi.com/point/daily"

querystring = {"lat": "40.77719879","lon": "-73.87259674","start":"2012-10-29","end":"2012-10-29"}
# https://meteostat.p.rapidapi.com/point/daily?lat=43.6667&lon=-79.4&start=2020-01-01&end=2020-01-31&alt=184'
headers = {
	"X-RapidAPI-Host": "meteostat.p.rapidapi.com",
	"X-RapidAPI-Key": ""
}
r = requests.request("GET", url, headers=headers, params=querystring)

print(r.text)
type(r)

{"meta":{"generated": "2022-05-06 08:32:48", "stations": ["KNYC0", "72502", "KTEB0", "74486"]},"data":[{"date":"2012-10-29","tavg":14.5,"tmin":12.2,"tmax":17.8,"prcp":15.2,"snow":0.0,"wdir":35.0,"wspd":54.7,"wpgt":null,"pres":null,"tsun":null}]}


requests.models.Response

In [47]:
import json

print(json.dumps(r.json(), indent=2))

{
  "meta": {
    "generated": "2022-05-06 08:32:48",
    "stations": [
      "KNYC0",
      "72502",
      "KTEB0",
      "74486"
    ]
  },
  "data": [
    {
      "date": "2012-10-29",
      "tavg": 14.5,
      "tmin": 12.2,
      "tmax": 17.8,
      "prcp": 15.2,
      "snow": 0.0,
      "wdir": 35.0,
      "wspd": 54.7,
      "wpgt": null,
      "pres": null,
      "tsun": null
    }
  ]
}


In [ ]:
querystring = {"lat": "40.77719879","lon": "-73.87259674","start":"2012-10-01","end":"2012-11-29"}

r = requests.request("GET", url, headers=headers, params=querystring)

print(r.text)
type(r)

In [71]:
# Set the url
url = "https://meteostat.p.rapidapi.com/point/daily"

# Set locations

points = {"DCA": ["38.852100", "-77.037697"],
	     "LGA": ["40.777199", "-73.872597"],
	     "IAD":	["38.944500", "-77.455803"],
	     "JFK":	["40.639801", "-73.778900"],
         "MDW":	["41.785999", "-87.752403"],
         "ORD":	["41.9786",	"-87.9048"],
         "LAX":	["33.94250107",	"-118.4079971"],
         "PHX":	["33.43429946899414", "-112.01200103759766"]}


# Create empty dataframe, will be used to append each location's weather data
weather_df = pd.DataFrame([])

# Loop through all locations
for key, value in points.items():
    # Create final url
    # Request data from url
    querystring = {"lat":value[0], "lon":value[1], "start":"2012-10-01", "end":"2012-11-29"}
    # https://meteostat.p.rapidapi.com/point/daily?lat=43.6667&lon=-79.4&start=2020-01-01&end=2020-01-31&alt=184'
    headers = {
    "X-RapidAPI-Host": "meteostat.p.rapidapi.com",
    "X-RapidAPI-Key": "06cc1bc58bmsh6b0e697965add32p1e5320jsnf3103b779dfa"
    }
    r = requests.request("GET", url, headers=headers, params=querystring)

    if r.status_code != 200:
        print(r.json())
        break
    
    import time 
    time.sleep(0.5) #uncomment if you run into a query limit
    
    # Decode repsonse with json decoder
    weather_temp = r.json()

    # Flatten json response
    weather_temp_df = pd.json_normalize(weather_temp, 
                                        sep="_", 
                                        record_path="data", 
                                        errors='ignore')
    
    # concatenate dataframes
    weather_df = pd.concat([weather_df, weather_temp_df], ignore_index=True)

# Print final dataset weather_df
weather_df

,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2012-10-01,16.9,12.2,22.2,0.0,0.0,NaN,5.8,None,1014.0,None
1,2012-10-02,20.3,18.3,22.2,10.9,0.0,NaN,7.9,None,1013.2,None
2,2012-10-03,23.8,21.1,27.8,0.0,0.0,NaN,4.7,None,1017.6,None
3,2012-10-04,23.9,18.9,27.2,0.3,0.0,NaN,7.6,None,1019.9,None
4,2012-10-05,21.8,16.1,28.3,0.0,0.0,NaN,7.9,None,1018.7,None
...,...,...,...,...,...,...,...,...,...,...,...
475,2012-11-25,20.1,13.9,27.8,0.0,None,NaN,5.0,None,1011.1,None
476,2012-11-26,18.8,12.8,26.7,0.0,None,NaN,6.1,None,1014.5,None
477,2012-11-27,19.9,12.8,28.9,0.0,None,NaN,5.8,None,1017.8,None
478,2012-11-28,19.9,13.9,26.7,0.0,None,NaN,6.1,None,1017.6,None
